In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
!{sys.executable} -m pip install emnist
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install sklearn

In [3]:
import numpy as np

def get_binary_vector_with_mean(mean, size):
    array_ones = np.ones((int(mean * size), 1))
    array_zeros = np.zeros((size - (int(mean * size)), 1))
    array = np.concatenate((array_ones, array_zeros), axis = 0)
    np.random.shuffle(array)
    return array

In [4]:
from shfl.core import data

array = get_binary_vector_with_mean(0.6, 10000)
print(len(array))
print(np.mean(array))

10000
0.6


In [5]:
from shfl.core.query import Query

class RandomizedQuery(Query):
    def get(self, array):
        coin_flip = np.random.rand()
        if (coin_flip > 0.5):
            return array
        second_coin_flip = np.random.rand()
        if (second_coin_flip > 0.5):
            return np.array([[1.0]])
        return np.array([[0.0]])

In [6]:
import core.federated_operation

def diff_privacy_mean_test(mean, data_size):
    array = get_binary_vector_with_mean(mean, data_size)
    federated_array = data.federate_array("my_federated_data", array, data_size)
    result = core.federated_operation.query_federated_data(federated_array, RandomizedQuery(), "my_federated_data")
    prob_head = 0.5
    p1_given_1 = prob_head + (1 - prob_head) * prob_head
    p1_given_0 = (1 - prob_head) * prob_head
    
    noise_removed_result = (np.mean(result) - p1_given_0)/(p1_given_1 - p1_given_0)
    
    print("Generated binary vector with mean: " + str(np.mean(array)))
    print("Differencial query mean result: " + str(np.mean(result)))
    print("Differencial query removing noise: " + str(noise_removed_result))
    print ("\n")

In [7]:
diff_privacy_mean_test(0.8, 10)
diff_privacy_mean_test(0.8, 100)
diff_privacy_mean_test(0.8, 1000)
diff_privacy_mean_test(0.8, 10000)
diff_privacy_mean_test(0.8, 100000)
diff_privacy_mean_test(0.8, 1000000)

Generated binary vector with mean: 0.8
Differencial query mean result: 0.9
Differencial query removing noise: 1.3


Generated binary vector with mean: 0.8
Differencial query mean result: 0.6
Differencial query removing noise: 0.7


Generated binary vector with mean: 0.8
Differencial query mean result: 0.661
Differencial query removing noise: 0.8220000000000001


Generated binary vector with mean: 0.8
Differencial query mean result: 0.6438
Differencial query removing noise: 0.7876000000000001


Generated binary vector with mean: 0.8
Differencial query mean result: 0.64984
Differencial query removing noise: 0.79968


Generated binary vector with mean: 0.8
Differencial query mean result: 0.649946
Differencial query removing noise: 0.799892


